## Importing the vertex amplitude as numpy array in Python

The EPRL vertex corresponds to the following object:

![alt text](../pics/vertex_amplitude.png "Title")

in which $j_{12} = j_{13} = j_{14} = \dots = j_{45} = j$,. 

At fixed $j$, the amplitude can be interpreted as a 5-dim array with dimensions $(2j+1)^5$, where each entry corresponds to different
values for the intertwiners $i_1 \dots i_5$ (notice that $k_5$ in the pic is a typo) . 
In fact, there are $2j+1$ possible values for each intertwiner (from $0$ to $2j$).

In the `../../data/EPRL_vertices/python` folder are provided the vertices for $j = 0.5, 1, \dots 5$, stored in `.npz` format.

In [1]:
import numpy as np

"""
Load vertex tensor as numpy multidimensional array
The vertex is returned in ROW-MAJOR (C-like) order
The vertex is read as [i5, i4, i3, i2, i1]
"""
def VertexLoad(spin, path="../../data/EPRL_vertices/Python/Dl_20"):

    assert (spin/0.5).is_integer(), "Please assign spin as float halfinteger"
    DIM = int(2*spin+1) 

    vertex_loaded = np.load(f"{path}/vertex_j_{float(spin)}.npz")

    if vertex_loaded.flags['F_CONTIGUOUS'] == False:
        vertex_loaded = np.transpose(vertex_loaded)     

    vertex = np.zeros((DIM,DIM,DIM,DIM,DIM), order='C')
    vertex[:] = vertex_loaded[:]
    
    return vertex


For example, we can load the 5-dimensional array corresponding to $j = 3$:

In [2]:
j = 3

vertex = VertexLoad(j)

In [3]:
vertex.data.contiguous

True

In [4]:
vertex.flags['F_CONTIGUOUS']

False

In [5]:
vertex.flags['C_CONTIGUOUS']

True

The amplitude $A \left( j, i_1 = 0, i_2 = 2, i_3 = 0, i_4 = 3, i_5 = 0 \right)$ correspons to the following element:

In [6]:
vertex[0,3,0,2,0]

-5.071973704515683e-13

The tensor is stored in row-major (C-like) order since we want to optimize the tensor contraction with the last index $i_1$

## Optimized contraction for star amplitude

In [20]:
def star_naive_contraction(tensor, indices, dim):

   result = 0.0

   for ib1 in range(dim):
      for ib2 in range(dim):
         for ib3 in range(dim):
            for ib4 in range(dim):
              for ib5 in range(dim):

                 result += tensor[ib5,ib4,ib3,ib2,ib1] \
                           * tensor[indices[0], indices[1], indices[2], indices[3], ib1] \
                           * tensor[indices[4], indices[5], indices[6], indices[7], ib2] \
                           * tensor[indices[8], indices[9], indices[10], indices[11], ib3] \
                           * tensor[indices[12], indices[13], indices[14], indices[15], ib4] \
                           * tensor[indices[16], indices[17], indices[18], indices[19], ib5]      

   return np.square(result)         

In [17]:
def star_optimized_contraction(tensor, indices, optimize_path=False):

    return np.square(np.einsum('abcde, e, d, c, b, a ->', tensor,  
                              tensor[indices[0], indices[1], indices[2], indices[3], :], 
                              tensor[indices[4], indices[5], indices[6], indices[7], :],
                              tensor[indices[8], indices[9], indices[10], indices[11], :],
                              tensor[indices[12], indices[13], indices[14], indices[15], :],
                              tensor[indices[16], indices[17], indices[18], indices[19], :],
                              optimize=optimize_path))

In [26]:
#tensor = np.random.rand(5,5,5)
vector = vertex[0,0,0,0,:]

indices = [0,1,4,3,4,0,1,4,3,2,0,1,4,3,0,0,1,4,3,0]

path_star_contraction = np.einsum_path('abcde, e, d, c, b, a->', vertex,vector,vector,vector,vector,vector, optimize='optimal')[0]

result_naive = star_naive_contraction(vertex, indices, int(2*j+1))
result_optimized = star_optimized_contraction(vertex, indices, optimize_path=path_star_contraction)

print(result_naive)
print(result_optimized)

4.979322253013476e-155
4.979322253013463e-155


The result is equal, but the optimized contraction is more than 2 orders of magnitude faster:

In [21]:
for iteration in range(1000):
    _ = star_naive_contraction(vertex, indices, int(2*j+1))

In [19]:
for iteration in range(1000):
    _ = star_optimized_contraction(vertex, indices, optimize_path=path_star_contraction)

On my laptop, the first cell takes $\sim 19$ seconds, the second one $\sim 0.1$ seconds.

## Computing the dihedral angle (diagonal operator)

If we want to compute the dihedral angle on one node, the equation is:

$$
\langle O_1 \rangle = \frac{1}{Z} \sum_{i_1 \dots i_5} A^2 \left( j, i_1, i_2, i_3, i_4, i_5 \right) f \left( j, i_1 \right)
$$

where:

$$
f \left( j, i_1 \right) = \frac{i_1(i_1 + 1) - 2j(j+1)}{2j(j+1)}
$$

is the (diagonal) matrix element, and:

$$
Z = \sum_{i_1 \dots i_5} A^2 \left( j, i_1, i_2, i_3, i_4, i_5 \right)
$$

is the normalization factor. Of course we have:

$$
\sum_{i_1 \dots i_5} \equiv \sum_{i_1=0}^{2j} \sum_{i_2=0}^{2j} \sum_{i_3=0}^{2j} \sum_{i_4=0}^{2j} \sum_{i_5=0}^{2j}
$$

The computation below is not efficient but we do not care in this context

In [3]:
def boundary_angle(i,j):
    angle = (i*(i+1) - 2*j*(j+1)) / (2*j*(j+1));
    return angle

exp_value = 0.0
norm_factor = 0.0

for i1 in range(0,2*j+1):
    
    angle = boundary_angle(i1, j)
    
    for i2 in range(0,2*j+1): 
        for i3 in range(0,2*j+1): 
            for i4 in range(0,2*j+1): 
                for i5 in range(0,2*j+1): 
                    
                    A_squared = pow(vertex[i1,i2,i3,i4,i5],2)
                    
                    exp_value += A_squared*angle
                    norm_factor += A_squared    
                    
print(f"Expectation value is {exp_value/norm_factor}")                    

Expectation value is -0.33333333333333365


# Joseph experiments

In [4]:
import tensorflow as tf


spin_j = j
vertex_amplitudes = tf.convert_to_tensor(
    vertex, dtype=tf.float64
)
squared_amplitudes = tf.math.square(vertex_amplitudes)
scale = tf.math.reduce_sum(squared_amplitudes)
# rewards = tf.cast(squared_amplitudes/scale, dtype=tf.float32)
rewards = squared_amplitudes/scale

2023-01-19 15:11:38.613782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
rewards.shape

TensorShape([7, 7, 7, 7, 7])

In [8]:
inds_1d = tf.range(7)
inds = tf.meshgrid(*[inds_1d]*5, indexing='ij')

i1 = tf.cast(inds[0], dtype=tf.float64)

In [9]:
angle = (i1*(i1 + 1) - 2*spin_j*(spin_j + 1)) / (2*spin_j*(spin_j + 1))

In [10]:
tf.reduce_sum(rewards*angle)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.33333333333333376>